<a href="https://colab.research.google.com/github/AdityaMohanty374/ML_Algos/blob/main/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
#in probability space

import pandas as pd
import numpy as np

df = pd.read_csv('diabetes.csv')
X = df.drop(columns = "Outcome").values
y = df['Outcome'].values
mu0 = X[y==0].mean(axis=0)
mu1 = X[y==1].mean(axis=0)
phi = np.mean(np.array(y)==1)
var0 = X[y==0].var(axis=0)
var1 = X[y==1].var(axis=0)
def naive_bayes(x, mu, var):
    n = x.shape[0]
    diff = x - mu
    exponent = np.exp(-0.5 * ((x-mu)**2)/(var))
    coeff = 1.0 / (np.sqrt(2 * np.pi*var))
    return np.prod(coeff * exponent)
predictions = []

for i in range(len(y)):
  x = X[i]
  px_y0 = naive_bayes(x, mu0, var0)
  px_y1 = naive_bayes(x, mu1, var1)
  posterior = (px_y1 * phi) / (px_y1 * phi + px_y0 * (1 - phi))
  predictions.append(1 if posterior>=0.5 else 0)
accuracy = 0
for i in range(len(predictions)):
  if predictions[i]==y[i]: accuracy+=1

accuracy/len(y)

0.7630208333333334

In [61]:
#in log-space
import pandas as pd
import numpy as np

df = pd.read_csv('diabetes.csv')
X = df.drop(columns="Outcome").values
y = df['Outcome'].values

mu0 = X[y==0].mean(axis=0)
mu1 = X[y==1].mean(axis=0)
phi = np.mean(y == 1)
var0 = X[y==0].var(axis=0) + 1e-9 #to avoid 0 variance
var1 = X[y==1].var(axis=0) + 1e-9

#Multiplying many small probabilities (np.prod) can lead to numbers too small for the computer to store → underflow to 0.0.
#Fix: work in log-space (np.sum of log-probabilities instead of product).

def naive_bayes_log(x, mu, var):
    coeff = -0.5 * np.log(2 * np.pi * var)
    exponent = -0.5 * ((x - mu) ** 2) / var
    return np.sum(coeff + exponent)

predictions = []
for i in range(len(y)):
    x = X[i]
    #In your current form, px_y1 * phi and px_y0 * (1 - phi) are likelihood × prior.
    #If you switch to log-space, you avoid computing posteriors directly — instead compare log-likelihoods:

    log_p0 = naive_bayes_log(x, mu0, var0) + np.log(1 - phi)
    log_p1 = naive_bayes_log(x, mu1, var1) + np.log(phi)
    predictions.append(1 if log_p1 > log_p0 else 0)

accuracy = np.mean(np.array(predictions) == y)
print("Accuracy:", accuracy)


Accuracy: 0.7630208333333334
